# On the fly Cholesky 

In [8]:
"""Build Benzene molecule with a minimal GTO basis, using PYSCF
"""
%load_ext autoreload
%autoreload 2

# Generic imports
import numpy as np
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
""" Compute wave functions, grid and density using PYSCF
"""
from isdf_prototypes.clean_isdf import benzene_from_pyscf

output_root = Path('../outputs/cholesky')
data: dict = benzene_from_pyscf(output_root, [10, 10, 10])
print("Returned data:", data.keys())
    
# Add real-space points, and volume element
data.update({'grid_points': data['cube_grid'].get_coords()})
data.update({'dV': data['cube_grid'].get_volume_element()})


converged SCF energy = -229.930646528207
Returned data: dict_keys(['wfs', 'rho', 'cube_grid'])


## Cholesky Algorithm Outline

This outlines the schematic pivoted Cholesky algorithm. Because the corresponding Gram matrix has shape $(np, np)$, we absolutely want to avoid storing this array in Octopus, so we use an on-the-fly version of the algorithm.

This will scale like $\mathcal{O}(n_p k N_e)$, where $n_p$ is the grid size, $k$ is the number of pivot points (which become our interpolation points), and $N_e$ is the number of occupied KS states (for molecular systems).

The generic expressions for the definition of $L$ can be found on [wikipedia](https://en.wikipedia.org/wiki/Cholesky_decomposition#The_Cholesky–Banachiewicz_and_Cholesky–Crout_algorithms)

This [paper](https://pubs.acs.org/doi/10.1021/acs.jctc.3c00615) gives a reference of CD w.r.t. ISDF, but the notation differs from the ISDF sources I follow.


### The Gram Matrix of the Pair Products

The Gram matrix for real systems is defined as:

$$
G_{r,p} = \sum_{ij} \phi_i(\mathbf{r}) \phi_j(\mathbf{r}) \phi_i(\mathbf{p}) \phi_j(\mathbf{r})
$$

where $r$ and $p$ are indices over grid points. In the on-the-fly algorithm, $p$ is defined as the pivot point, and so, one only computes the element $G_{r,p}$ per grid point of the inner loop (see below)

Because the pair products have a separable form, this can be rewritten as:

$$
G_{r,p} = \left[\sum_{i} \phi_i(\mathbf{r}) \phi_i(\mathbf{p}) \right] 
   \left[\sum_{j}  \phi_j(\mathbf{r}) \phi_j(\mathbf{p}) \right]
$$

reducing the scaling from $\mathcal{O}(N^2_e)$ to $\mathcal{O}(N_e)$ per grid point $r$.

The diagonal of the Gram matrix simplifies to:

$$
G_{r,r} = G_{r} = \left[\sum_{i} \phi_i(\mathbf{r})^2 \right] 
   \left[\sum_{j}  \phi_j(\mathbf{r})^2 \right]
$$


### Algorithm Outline

Note, fortran is column-major and python (numpy) is row-major. The fastest index will swap when moving from python to fortran.

```text
subroutine pivtoed_cholsky_on_the_fly(st, k_max, thres, pivots):
    
    k_max <- Maximum number of pivot points
    Allocate L(np, k_max), diag_G(np), pivots(k_max), residuals(np)

    ! Initialise diagonal of the Gram matrix
    do ip = 1, np
       ! Implement according to the expression above
       diag_G(ip) = gram_matrix_diagonal(st, n_occ)
    enddo

    do k = 1, k_max
        ! Choose the pivot index
        i_pivot = argmax(diag_G)
        pivots(k) = diag_G(i_pivot)
        if (diag_G(i_pivot) < thres) return pivots(1:k)

        ! Set the pivot value of the L matrix
        L(i_pivot, k) = sqrt(diag_G(i_pivot))
        inv_L = 1.0 / L(i_pivot, k) 

        ! Define the residuals
        if k == 1:
          residuals = 0.0
        else
          DGEMV L(1:np, 1:k-1), transpose(L(i_pivot, 1:k-1))
          -> residual(1:np)
        endif

        ! Set all other rows of column k
        do ip = 1, np
          ! Avoid overwriting the above, L(i_pivot, k)
          if ip == i_pivot: cycle
            
          ! Compute G on-the-fly, as defined in the expression above
          G_rp = gram_matrix_element(ip, i_pivot, st)

          L(ip, k) = inv_L * (G_rp - residual(ip)) 
          
          ! Update the diagonal
          diag_G(ip) = diag_G(ip) - L(ip, k)**2
        enddo
   enddo

end subroutine     
```

Once done, we can use the existing code in this repository to construct the ISDF vectors, and quantify the error w.r.t. the exact wave function product.


In [11]:
""" Cholesky implementation
"""
# Total number of grid points
np = data['grid_points'].shape[0]
# Total number of states
n_states = data['wfs'].shape[1]
assert data['wfs'].shape == (np, n_states), "Wavefunctions (MOs) array has shape (np, n_states)"

# Implement the above algorithm